<a href="https://colab.research.google.com/github/Medha-Shridharan/Bayesian-Optimization_Split-LAMP-primers/blob/main/reagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install botorch
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 481 kB 5.9 MB/s 
     |████████████████████████████████| 245 kB 66.6 MB/s 
     |████████████████████████████████| 152 kB 55.6 MB/s 
     |████████████████████████████████| 727 kB 63.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from botorch.acquisition import ConstrainedExpectedImprovement



# # train_X = torch.rand(10, 2)
# Y = 1 - torch.norm(train_X - 0.5, dim=-1, keepdim=True)
# # Y = Y + 0.1 * torch.randn_like(Y)  # add some noise
# # train_Y = standardize(Y)

#train_Y dataset is (r**2)*(1/gradient)

#train_X = torch.log10(train_X)

#ALL OF THIS IS TRIPLICATE DATA

train_X = torch.tensor([[100, 100, 100, 100, 100],
                        [100, 100, 100, 100, 10],
                        [100, 100, 100, 100, 1e-8],
                        [100, 100, 100, 1600, 1e-8],
                        [100, 100, 100, 160, 1e-8],
                        [100, 100, 100, 16, 1e-8],
                        [100, 100, 100, 1.6, 1e-8],
                        [100, 100, 100, 0.16, 1e-8],
                        [160, 100, 100, 100, 1e-8],
                        [16, 100, 100, 100, 1e-8],
                        [1.6, 100, 100, 100, 1e-8],
                        [0.16, 100, 100, 100, 1e-8],
                        [15, 165, 180, 175, 30],
                        [75, 120, 165, 155, 35],
                        [90, 195, 195, 65, 10],
                        [90, 80, 40, 120, 60],
                        [75, 100, 70, 25, 125]])

train_Y = torch.tensor([[5.70371488],
                        [6.20255544],
                        [6.30610722],
                        [3.91294488],
                        [5.76402723],
                        [7.06502575],
                        [7.88795961],
                        [6.83329838],
                        [3.975726],
                        [7.03076466],
                        [2.4859599],
                        [0.034931],
                        [1.70840853],
                        [3.436176],
                        [3.114558],
                        [5.652],
                        [6.340245]])


gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

ModuleNotFoundError: ignored

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
# constraints = {0: (0.0, 200), 1: (10,400), 2: (10,400),3:(10,400),4:(10,400)}
# best_f = 30
# cEI = ConstrainedExpectedImprovement(gp, best_f,constraints)

from botorch.acquisition import qUpperConfidenceBound

UCB = qUpperConfidenceBound(gp, beta=0.1)




In [ ]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.tensor([1e-8,10.0, 10.0, 10.0, 10.0]), torch.tensor([200, 400, 400, 400, 400])])

# for i in range(len(bounds)):
#   bounds[i] = torch.log10(bounds[i])

candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=5, num_restarts=5, raw_samples=20,
)


# 10**(candidate)
candidate

In [ ]:


# reducing noise in the experiments
# pipetting?
# try pushing the left curve more to the left can add a (10-x) term to the y value output where x is the ct value of the 160 curve
# try triplicates?

# why didnt the last models work: pitfalls
# they just didnt know what to ask for?
# not enough datapoints for computer to fit the model (eg 0.0016 is too low same as 0, 1600 is too high close to 160) -> not enough difference to get response in output

# is this still worth trying
# maybe if can find a good y variable to fit to

# graphs: use originlab
# drawings: use autodraw

# - can I stay beyond hols (do I still need clearance form)
# try b2 and f2 5x5 grid

# assumptions of BO:
# - points close together will have similar values of output (continuous function)


# things to put in the discussion of ur report:
# - ct shift can only be detected in the log range (100v1 change not 2v1 change)
# - LAMP system is very complex and does not work based on a linear relationship between concentration variables across ranges
# - this project acts as POC that andgate is possible (2 inputs to come up with good output signal) -> to be optimized further, need to decide on an rna, decide on amplification method,
# decide on optimized results

# next steps in writing the report:
# - do triplicates for 160, 16, 1.6, 0.16 for the necessary F2 range
# - input this into the model to see if it outputs any reasonable new points in the space to try

# mistakes in the ssef application:
# - title should be AND gate NOT endgate
# - teacher in charge is maury julien jean pierre (with appropriate email) NOT fernandez



